# 기초세팅

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Colab\ Notebooks/IndustrialClassification

/content/drive/MyDrive/Colab Notebooks/IndustrialClassification


## Import

In [ ]:
import pickle
import pandas as pd
import seaborn as sns
import numpy as np
from typing import *
from gensim.corpora import Dictionary

# Data load

In [ ]:
df = pd.read_csv('data/dataset.csv', index_col='AI_id')

In [ ]:
df

,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
AI_id,,,,,,
id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매
id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치
id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육
...,...,...,...,...,...,...
id_0999996,C,13,134,제품입고,워싱,청바지워싱
id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어
id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매


In [ ]:
df_test = pd.read_csv('data/testset.csv', index_col='AI_id')

In [ ]:
df_test

,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
AI_id,,,,,,
id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...
id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매


# Spell

In [45]:
from hanspell import spell_checker

In [46]:
result = spell_checker.check(u'안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다.')

In [47]:
result.as_dict()

{'checked': '안녕하세요. 저는 한국인입니다. 이 문장은 한글로 작성됐습니다.',
 'errors': 4,
 'original': '안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다.',
 'result': True,
 'time': 0.837113618850708,
 'words': OrderedDict([('안녕하세요.', 2),
              ('저는', 0),
              ('한국인입니다.', 2),
              ('이', 2),
              ('문장은', 2),
              ('한글로', 0),
              ('작성됐습니다.', 1)])}

# External tokenizer

## SKT

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.0 MB/s 
     |████████████████████████████████| 895 kB 59.1 MB/s 
     |████████████████████████████████| 6.5 MB 32.9 MB/s 
     |████████████████████████████████| 596 kB 55.0 MB/s 
     |████████████████████████████████| 67 kB 4.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import PreTrainedTokenizerFast

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [ ]:
type(df.text_obj[0])

str

In [ ]:
tokenizer.tokenize(df.text_obj[2])

['▁절단', '하여', '사업', '체', '에도', '매']

In [ ]:
tokenizer.tokenize('절단하여 사업체에 도매')

['▁절단', '하여', '▁사업', '체에', '▁도매']

In [ ]:
def report_error(s: str) -> None:
  try:
    tokenizer.tokenize(s)
  except:
    print(s)

In [ ]:
def tokenize(s: str)-> List[str]:
  try:
    return tokenizer.tokenize(s)
  except:
    return []

In [ ]:
df['text_obj_skt'] = df.text_obj.apply(tokenize)

In [ ]:
df['text_mthd_skt'] = df.text_mthd.apply(tokenize)
df['text_deal_skt'] = df.text_deal.apply(tokenize)

In [ ]:
df

,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text_obj_skt,text_mthd_skt,text_deal_skt
AI_id,,,,,,,,,
id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,"[▁카, 센터, 에서]","[▁자동차, 부분, 정비]","[▁타이어, 오, 일, 교환]"
id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,"[▁상점, 내에서]","[▁일반, 인을, ▁대상으로]","[▁채소, ., 과일, 판매]"
id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,"[▁절단, 하여, 사업, 체, 에도, 매]","[▁공업, 용, 고, 무를, 가, 지고]","[▁합성, 고, 무도, 매]"
id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,"[▁영업, 점에서]","[▁일반, 소비, 자에게]","[▁열쇠, 잠, 금, 장치]"
id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,"[▁어린이, 집]","[▁보호, 자의, ▁위탁, 을, ▁받아]","[▁취학, 전, 아동, 보육]"
...,...,...,...,...,...,...,...,...,...
id_0999996,C,13,134,제품입고,워싱,청바지워싱,"[▁제품, 입, 고]","[▁워, 싱]","[▁청, 바지, 워, 싱]"
id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어,[▁현장에서],"[▁고객의, 요, 청에, 의, 해]","[▁실내, 인, 테, 리어]"
id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매,"[▁영업, 점에서]","[▁일반, 소비, 자에게]","[▁여성의, 류, ▁판매]"


In [ ]:
df.text_obj_skt.apply(len).value_counts()

2     508860
3     217174
4      86034
1      75268
5      53790
0      16677
6      15580
7       9092
8       5807
9       4035
10      2567
11      1703
12      1121
13       853
14       507
15       374
16       191
17       131
18        92
19        58
20        38
21        14
22        14
24         5
25         4
26         3
23         3
27         2
29         1
33         1
28         1
Name: text_obj_skt, dtype: int64

In [ ]:
df.text_mthd_skt.apply(len).value_counts()

3     285968
4     228448
2     194117
5      86119
6      76793
0      43619
1      39528
7      19246
8      11515
9       7307
10      2892
11      1766
12      1053
13       614
14       352
15       215
16       156
17       102
18        65
19        34
20        30
21        17
23        16
22        13
24         4
25         3
26         3
30         2
27         1
34         1
28         1
Name: text_mthd_skt, dtype: int64

## Konlpy

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 6.7 MB/s 
     |████████████████████████████████| 448 kB 66.0 MB/s 


In [ ]:
from konlpy.tag import Kkma, Komoran, Okt

In [ ]:
class IndustrialTokenizer:
  def __init__(self,
               tager,
               token_dictionary_fname='data/token_dictionary_220326.dic'):
    self.changer = {'a/s':' 수리 ', '써비스':'서비스', '짜장면':'자장면', '제단':'재단', '맛사지':'마사지', '마시지': '마사지', '프라스틱':'플라스틱', '돌보미':'돌봄'}
    self.separators = ['+', '-', '[', ']', '>', '<', '/', 'ㆍ', 'ㆍ',':', ';', '?', '√', '？', '&', ',', '，', '.', '．', '(', ')', '및']
    self.tager = tager
    self.dictionary = Dictionary.load(token_dictionary_fname)

  def get_pre_tokens(self, fname: str) -> List[str]:
    return list(pd.read_csv(fname)['0']).sort(key=len, reverse=True)

  def tokenize_and_encode(self, s: str) -> List[int]:
    return self.encode(self.tokenize(s))

  def encode(self, tokens: List[str]) -> List[int]:
    return list(map(lambda i: i + 2, self.dictionary.doc2idx(tokens)))

  def tokenize(self, s: str) -> List[str]:
    try:
      return self.try_tokenize(s)
    except:
      print(s)
      return []

  def try_tokenize(self, s: str) -> List[str]:
    return list(map(lambda t: t[0], self.tager.pos(self.nomalize(s)))) if pd.notna(s) else []

  def nomalize(self, s: str) -> str:
    return self.separate(self.change(s))

  def separate(self, s: str) -> str:
    for seq in self.separators:
      s = s.replace(seq, ' ')
    return s

  def change(self, s: str) -> str:
    for asis, tobe in self.changer.items():
      s = s.replace(asis, tobe)
    return s


### Kkma

In [ ]:
tokenizer = IndustrialTokenizer(Kkma())

In [ ]:
df['obj_kkma'] = df.text_obj.apply(tokenizer.tokenize)

In [ ]:
df['mthd_kkma'] = df.text_mthd.apply(tokenizer.tokenize)

In [ ]:
df['deal_kkma'] = df.text_deal.apply(tokenize)

### Komoran

In [ ]:
tokenizer = IndustrialTokenizer(Komoran())

#### Data set

In [ ]:
df['obj_komoran'] = df.text_obj.apply(tokenizer.tokenize)

In [ ]:
df['mthd_komoran'] = df.text_mthd.apply(tokenizer.tokenize)

In [ ]:
df['deal_komoran'] = df.text_deal.apply(tokenizer.tokenize)

？
？
？


In [ ]:
df.to_csv('data/komoran_dataset.csv')

In [ ]:
with open('data/komoran_dataset.pkl', 'wb') as f:
  pickle.dump(df, f)

In [36]:
with open('data/komoran_dataset.pkl', 'rb') as f:
  df = pickle.load(f)

In [ ]:
df.obj_komoran.explode().value_counts()

에서        603450
사업장       117972
로          85787
매장         75932
음식점        69917
           ...  
하이초등학교         1
다중             1
화정             1
매장에ㅅ           1
퀼팅구조체          1
Name: obj_komoran, Length: 19491, dtype: int64

In [ ]:
df.mthd_komoran.explode().value_counts()

을           288992
으로          244473
대상          230377
고           158151
고객          156007
             ...  
토로               1
호드               1
부동산중개소엣어         1
아사이베리            1
란시               1
Name: mthd_komoran, Length: 18154, dtype: int64

In [ ]:
df.deal_komoran.explode().value_counts()

서비스        172942
판매         102505
운송          88124
화물          49016
제공          48753
            ...  
부인병             1
오카리나팬플릇         1
하그              1
골프연숩장           1
트리스             1
Name: deal_komoran, Length: 22294, dtype: int64

In [37]:
doc = list(df.obj_komoran) + list(df.mthd_komoran) + list(df.deal_komoran)

In [38]:
dictionary = Dictionary(doc)

In [ ]:
len(dictionary)

39049

In [39]:
dictionary.filter_extremes(no_below=2, no_above=1.0)

In [40]:
len(dictionary)

18928

#### Test set

In [ ]:
df_test['obj_komoran'] = df_test.text_obj.apply(tokenizer.tokenize)

In [ ]:
df_test['mthd_komoran'] = df_test.text_mthd.apply(tokenizer.tokenize)

In [ ]:
df_test['deal_komoran'] = df_test.text_deal.apply(tokenizer.tokenize)

？


In [ ]:
df_test.to_csv('data/komoran_testset.csv')

In [ ]:
with open('data/komoran_testset.pkl', 'wb') as f:
  pickle.dump(df_test, f)

In [ ]:
df_test.obj_komoran.explode().value_counts()

에서      58777
사업장     15219
매장       7374
을        4532
음식점      3988
        ...  
구독          1
국방          1
동인천역        1
쌍수          1
태국          1
Name: obj_komoran, Length: 8523, dtype: int64

In [ ]:
df_test.mthd_komoran.explode().value_counts()

대상        20835
을         19793
으로        17547
에게        13387
고객        13318
          ...  
쇼프트와이딩        1
샘             1
미터기           1
졉객시설을         1
딩기            1
Name: mthd_komoran, Length: 8222, dtype: int64

In [ ]:
df_test.deal_komoran.explode().value_counts()

판매          10884
서비스          7391
등            4631
소매           4258
제공           4142
            ...  
박스ㅏ제조           1
홍삼액기스등판매        1
자연과학            1
실종              1
채화              1
Name: deal_komoran, Length: 10672, dtype: int64

In [ ]:
doc = list(df_test.obj_komoran) + list(df_test.mthd_komoran) + list(df_test.deal_komoran)

In [ ]:
dictionary = Dictionary(doc)

In [ ]:
len(dictionary)

16274

In [ ]:
dictionary.filter_extremes(no_below=2, no_above=1.0)

In [ ]:
len(dictionary)

9176

### Okt

In [ ]:
tokenizer = IndustrialTokenizer(Okt())

#### Data set

In [ ]:
df['obj_komoran'] = df.text_obj.apply(tokenizer.tokenize)

In [ ]:
df['mthd_komoran'] = df.text_mthd.apply(tokenizer.tokenize)

In [ ]:
df['deal_komoran'] = df.text_deal.apply(tokenizer.tokenize)

In [ ]:
df.to_csv('data/okt_dataset.csv')

In [ ]:
with open('data/okt_dataset.pkl', 'wb') as f:
  pickle.dump(df, f)

In [ ]:
with open('data/okt_dataset.pkl', 'rb') as f:
  df = pickle.load(f)

In [ ]:
df.obj_komoran.explode().value_counts().to_csv('okt_obj_tokens.csv')

In [ ]:
df.mthd_komoran.explode().value_counts().to_csv('okt_mthd_tokens.csv')

In [ ]:
df.deal_komoran.explode().value_counts().to_csv('okt_deal_tokens.csv')

In [ ]:
doc = list(df.obj_komoran) + list(df.mthd_komoran) + list(df.deal_komoran)

In [ ]:
dictionary = Dictionary(doc)

In [ ]:
len(dictionary)

39049

In [ ]:
dictionary.filter_extremes(no_below=2, no_above=1.0)

In [ ]:
len(dictionary)

21728

#### Test set

In [ ]:
df_test['obj_komoran'] = df_test.text_obj.apply(tokenizer.tokenize)

In [ ]:
df_test['mthd_komoran'] = df_test.text_mthd.apply(tokenizer.tokenize)

In [ ]:
df_test['deal_komoran'] = df_test.text_deal.apply(tokenizer.tokenize)

In [ ]:
df_test.to_csv('data/okt_testset.csv')

In [ ]:
with open('data/okt_testset.pkl', 'wb') as f:
  pickle.dump(df_test, f)

In [41]:
df_test.obj_komoran.explode().value_counts().to_csv('okt_test_obj_tokens.csv')

In [42]:
df_test.mthd_komoran.explode().value_counts().to_csv('data/okt_test_mthd_tokens.csv')

In [43]:
df_test.deal_komoran.explode().value_counts().to_csv('data/okt_test_deal_tokens.csv')

In [ ]:
doc = list(df_test.obj_komoran) + list(df_test.mthd_komoran) + list(df_test.deal_komoran)

In [ ]:
dictionary = Dictionary(doc)

In [ ]:
len(dictionary)

17972

In [ ]:
dictionary.filter_extremes(no_below=2, no_above=1.0)

In [ ]:
len(dictionary)

10052

# Custom Tokenizer

In [ ]:
class DealTokenizer:
  def __init__(self,
               pre_tokens_fname='data/pre_toks_220321_4.csv',
               token_dictionary_fname='data/token_dictionary.dic'):
    self.changer = {'a/s':' 수리 ', '써비스':'서비스', '짜장면':'자장면', '제단':'재단', '맛사지':'마사지', '마시지': '마사지', '프라스틱':'플라스틱', '돌보미':'돌봄'}
    self.separators = ['+', '-', '[', ']', '>', '<', '/', 'ㆍ',':', ';', '?', '？', '&', ',', '，', '.', '．', '(', ')', '및']
    self.pre_tokens = self.get_pre_tokens(pre_tokens_fname)
    self.dictionary = Dictionary.load(token_dictionary_fname)

  def get_pre_tokens(self, fname: str) -> List[str]:
    return list(pd.read_csv(fname)['0']).sort(key=len, reverse=True)

  def tokenize_and_encode(self, s: str) -> List[int]:
    return self.encode(self.tokenize(s))

  def encode(self, tokens: List[str]) -> List[int]:
    return list(map(lambda i: i + 2, self.dictionary.doc2idx(tokens)))

  def tokenize(self, s: str) -> List[str]:
    return self.replace(s).split() if s is not np.nan else []

  def replace(self, s: str) -> str:
    return self.extract_pre_tokens(self.separate(self.change(s)))

  def extract_pre_tokens(self, s: str) -> str:
    for token in self.pre_tokens:
      s = s.replace(token, f' {token} ')
    return s

  def separate(self, s: str) -> str:
    for seq in self.separators:
      s = s.replace(seq, ' ')
    return s

  def change(self, s: str) -> str:
    for asis, tobe in self.changer.items():
      s = s.replace(asis, tobe)
    return s


## Build pre_token

### Default

In [ ]:
tokenizer = DealTokenizer()

In [ ]:
toks = df.text_deal.apply(tokenizer.tokenize)

In [ ]:
toks

AI_id
id_0000001      [타이어, 오일, 교환]
id_0000002       [채소, 과일, 판매]
id_0000003     [합성, 고, 무도, 매]
id_0000004     [열, 쇠, 잠금, 장치]
id_0000005      [취학전, 아동, 보육]
                   ...       
id_0999996        [청, 바지, 워싱]
id_0999997         [실내, 인테리어]
id_0999998     [여성, 의, 류, 판매]
id_0999999             [필라테스]
id_1000000    [한식, 미역, 구, 판매]
Name: text_deal, Length: 1000000, dtype: object

In [ ]:
toks.to_csv('data/toks_test.csv')

In [ ]:
mthd_toks = df.text_mthd.apply(tokenizer.tokenize)

In [ ]:
mthd_toks.to_csv('data/mthd_toks_test.csv')

In [ ]:
toks = toks.explode().value_counts()

In [ ]:
toks[toks > 100]

서비스     174101
판매      103486
운송       88627
제공       49587
화물       47635
         ...  
로또복권       101
냉동기        101
발레         101
내부         101
장치         101
Name: text_deal, Length: 1610, dtype: int64

In [ ]:
toks.to_csv('data/toks_v1.csv')

### Add pre_toks

In [ ]:
pre_toks = list(toks[toks > 100].index)

In [ ]:
tokenizer.pre_tokens += pre_toks

In [ ]:
new_toks = df.text_deal.apply(tokenizer.tokenize).explode().value_counts()

In [ ]:
new_toks

서비스       174101
판매        103486
운송         88627
기          86702
물          81425
           ...  
규결화한           1
노동자이익향         1
봉투만듬           1
대생             1
양채류-양          1
Name: text_deal, Length: 30583, dtype: int64

In [ ]:
new_toks.to_csv('data/toks_v2.csv')

### Refine pre_toks

In [ ]:
pre_toks = pd.read_csv('data/toks_v3.csv')

In [ ]:
pre_toks = list(pre_toks.token)

In [ ]:
tokenizer = DealTokenizer()

In [ ]:
tokenizer.pre_tokens += pre_toks

In [ ]:
new_toks = df.text_deal.apply(tokenizer.tokenize).explode().value_counts()

In [ ]:
new_toks

서비스         174101
판매          103486
운송           88627
제공           49587
화물           49197
             ...  
모니터사용설명서         1
돼고고기             1
푸전               1
SPINDLE          1
용매트리스            1
Name: text_deal, Length: 43696, dtype: int64

In [ ]:
new_toks.to_csv('data/tokes_v4.csv')

### Use industial class

In [ ]:
df_class = pd.read_csv('data/ind_class.csv')

In [ ]:
df_class

,col1,col2,col3,col4
0,농업,작물 재배업,곡물 및 기타 식량작물 재배업,곡물 및 기타 식량작물 재배업
1,NaN,NaN,"채소, 화훼작물 및 종묘 재배업",채소작물 재배업
2,NaN,NaN,NaN,화훼작물 재배업
3,NaN,NaN,NaN,종자 및 묘목 생산업
4,NaN,NaN,"과실, 음료용 및 향신용 작물 재배업",과실작물 재배업
...,...,...,...,...
1192,가구 내 고용활동,가구 내 고용활동,가구 내 고용활동,가구 내 고용활동
1193,달리 분류되지 않은 자가 소비를 위한 가구의 재화 및 서비스 생산활동,자가 소비를 위한 가사 생산 활동,자가 소비를 위한 가사 생산 활동,자가 소비를 위한 가사 생산 활동
1194,NaN,자가 소비를 위한 가사 서비스 활동,자가 소비를 위한 가사 서비스 활동,자가 소비를 위한 가사 서비스 활동
1195,국제 및 외국기관,국제 및 외국기관,국제 및 외국기관,주한 외국 공관


In [ ]:
cs = list(df_class.col1.dropna()) + list(df_class.col2.dropna()) + list(df_class.col3.dropna()) + list(df_class.col4.dropna())

In [ ]:
df_class = pd.DataFrame(cs)

In [ ]:
df_class

,0
0,농업
1,임업
2,어업
3,"석탄, 원유 및 천연가스 광업"
4,금속 광업
...,...
1995,가구 내 고용활동
1996,자가 소비를 위한 가사 생산 활동
1997,자가 소비를 위한 가사 서비스 활동
1998,주한 외국 공관


In [ ]:
tokenizer = DealTokenizer()

In [ ]:
toks = df_class[0].apply(tokenizer.tokenize).explode().value_counts()

In [ ]:
toks

업       1221
제조       688
기타       359
제품       189
서비스      159
        ... 
너트류        1
나사         1
캔          1
포장용기       1
공관         1
Name: 0, Length: 1551, dtype: int64

In [ ]:
toks.to_csv('data/ind_class_toks.csv')

#### Ver 2

In [ ]:
toks.Column1

,Column1
0,제조
1,기타
2,제품
3,서비스
4,도매
...,...
1687,너트
1688,나사
1689,캔
1690,포장


In [ ]:
ind_class_toks = pd.read_csv('data/ind_class_toks.csv')

In [ ]:
'합성고무' in ind_class_toks['Unnamed: 0']

False

In [ ]:
compounds = []

In [ ]:
prefixes = ['비']
subfixes = ['업', '류', '식', '품', '용', '물', '제', '학', '장', '방', '관', '화', '과', '기', '소', '점', '원', '터', '별', '교', '사']

for t in ind_class_toks['Unnamed: 0']:
  if len(t) < 2:
    continue
  
  if sum(map(t.startswith, prefixes)) + sum(map(t.endswith, subfixes)) > 0:
    compounds.append(t)



In [ ]:
for t in ind_class_toks['Unnamed: 0']:
  if t.startswith('재'):
    print(t)

재배업
재생업
재산권
재생
재료
재
재보험업
재처리업
재생용
재화
재활원
재정
재해


In [ ]:
#len(compounds)

586

In [ ]:
len(compounds)

1311

In [ ]:
toks_v2 = list(pd.read_csv('data/ind_class_toks_v2.csv').Column1)

In [ ]:
toks_v2 += compounds

In [ ]:
len(toks_v2)

2345

In [ ]:
toks_v2 = list(set(toks_v2))

In [ ]:
len(toks_v2)

1617

In [ ]:
pre_toks = list(pd.read_csv('data/toks_v3.csv').token)

In [ ]:
pre_toks = list(set(pre_toks))

In [ ]:
len(pre_toks)

875

In [ ]:
pre_toks += toks_v2

In [ ]:
pre_toks = list(set(pre_toks))

In [ ]:
pre_toks += tokenizer.pre_tokens

In [ ]:
pre_toks = list(set(pre_toks))

In [ ]:
len(pre_toks)

2120

In [ ]:
pd.DataFrame(pre_toks).to_csv('data/pre_toks_220321_2.csv', index=False)

In [ ]:
pt = pd.read_csv('data/pre_toks_220321_2.csv')

In [ ]:
for t in tokenizer.pre_tokens:
  if len(t) == 1:
    print(t)

회
류
열
을
탭
쌀
캔
꽃
솜
펌
닭
차
찜
초
밥
술
벽
컷
죽
콩
약
못
댐
등
옷
를
티
떡
석
톱
책
침
즙
빵


In [ ]:
pt[pt['0'].apply(len) > 1].to_csv('data/pre_toks_220321_3.csv', index=False)

In [ ]:
pt_over_1 = list(pt[pt['0'].apply(len) > 1]['0'])

In [ ]:
len(pt_over_1)

2086

In [ ]:
'내장' in pt_over_1

False

In [ ]:
pt_over_1.remove('내장')

In [ ]:
pt_over_1 += ['일본', '양성', '혈압', '패널']

In [ ]:
pt = pt_over_1 + ['을', '를', '등', '댐', '떡', '옷', '컷', '꽃', '류', '밥', '못', '즙', '빵']

In [ ]:
pd.DataFrame(pt).to_csv('data/pre_toks_220321_4.csv', index=False)

### Mthd

In [ ]:
pt = list(pd.read_csv('data/pre_toks_220321_4.csv')['0'])

In [ ]:
'업자' in pt

False

In [ ]:
new_pt = ['2차', '3차', '매장', '들','업자','이용자', '중학생', '초등학생', '고등학생', '대학생', '대학원생', '직장인', '직장', '사용자', '사용', '보호자', '일반인들', '보호자들', '체결', '체인', '금은','에의해', '위탁', '받아', '받는', '의해','접객', '고객', '으로', '로', '으로서', '로서','으로써', '로써', '에게', '대상', '학생들', '학생', '현장', '에서', '께서', '한테', '한테서', '더러', '에게서', '처럼', '같이', '만큼', '만치', '하고', '이며', '에다', '에다가', '부터', '로부터', '으로부터', '까지', '마저', '조차', '일랑', '커녕', '뿐', '뿐만', '아니라', '따라', '토록', '밖에', '대로', '데로', '든가', '든지', '이며', '보다', '처럼', '만큼', '같이', '조차', '마저', '까지', '이나', '이나마', '나마', '이라도', '라도', '하여']

In [ ]:
new_pt = list(set(new_pt))

In [ ]:
len(new_pt)

78

In [ ]:
pt += new_pt

In [ ]:
len(pt)

2224

In [ ]:
pt = list(set(pt))

In [ ]:
len(pt)

2180

In [ ]:
pt.sort(key=len, reverse=True)

In [ ]:
pd.DataFrame(pt).to_csv('data/pre_toks_220322_1.csv', index=False)

In [ ]:
tokenizer = DealTokenizer(pre_tokens='data/pre_toks_220322_1.csv')

In [ ]:
mthd_toks = df.text_mthd.apply(tokenizer.tokenize)

In [ ]:
mthd_toks.to_csv('data/mthd_toks_test_220322.csv')

In [ ]:
mthd_toks_cnt = mthd_toks.explode().value_counts()

In [ ]:
mthd_toks_cnt.to_csv('data/mthd_toks_cnt_220322.csv')

In [ ]:
mthd_toks_cnt

을         291219
로         273038
으         245805
대상        230920
일반        227086
           ...  
광주e7t          1
Tworld         1
멸종위기동          1
글렌딩            1
예채             1
Name: text_mthd, Length: 29103, dtype: int64

In [ ]:
'카페' in pt

False

In [ ]:
new_pt = ['카페','입찰','하도급','위기','기업','외래','행위','영업','무도시설', '도시민', '농민', '없이', '의뢰', '의뢰인', '계통', '사찰', '세액', '계산','요청', '인해', '중고생', '관할', '지역', '구청', '교육청', '인가', '간에', '중탕', '받음', '통해', '통해서', '다른', '위기', '수거', '몰딩', '에폭시', '관해서', '관하여', '해서', '하여서', '부착', '레스토랑', '초음파', '매입', '카메라', '사출', '조임', '프레임', '아래', '없는', '자판기', '미만', '가게', '국악', '갖추고', '업체', '화원', '방지', '교회','향상','수수료', '시설', '하면', '하면서', '벽면', '요구', '고객', '수퍼', '의거', '의해서', '의한', '의하면']

In [ ]:
new_pt = list(set(new_pt))

In [ ]:
risky_pt = ['에']

In [ ]:
to_del_pt = ['도시']

In [ ]:
pt.remove('도시')

In [ ]:
pt += new_pt

In [ ]:
pt += risky_pt

In [ ]:
pt = list(set(pt))

In [ ]:
pt.sort(key=len, reverse=True)

In [ ]:
pd.DataFrame(pt).to_csv('data/pre_toks_220322_2.csv', index=False)

### Obj

In [ ]:
tokenizer = DealTokenizer('data/pre_toks_220322_2.csv')

In [ ]:
obj_toks = df.text_obj.apply(tokenizer.tokenize)

In [ ]:
obj_toks.to_csv('data/obj_toks_test.csv')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [ ]:
obj_toks

AI_id
id_0000001               [카센터, 에, 서]
id_0000002               [상점내, 에, 서]
id_0000003    [절단, 하여, 사업, 체, 에, 도매]
id_0000004             [영업, 점, 에, 서]
id_0000005                  [어린이, 집]
                       ...          
id_0999996                  [제품, 입고]
id_0999997                [현장, 에, 서]
id_0999998             [영업, 점, 에, 서]
id_0999999             [사업, 장, 에, 서]
id_1000000             [사업, 장, 에, 서]
Name: text_obj, Length: 1000000, dtype: object

In [ ]:
obj_toks_cnt = obj_toks.explode().value_counts()

In [ ]:
obj_toks_cnt.to_csv('data/obj_toks_cnt.csv')

In [ ]:
pt = list(pd.read_csv('data/pre_toks_220322_2.csv')['0'])

In [ ]:
'사업체' in pt

False

In [ ]:
new_pt = ['사업체','목욕탕', '측정기', '공부', '공방', '고교', '매점', '지회', '지부', '총연합회', '연합회', '양조장', '양조', '프랜차이즈', '성경', '아카데미', '집계', '현황', '횟집', '한국']

In [ ]:
to_del = ['조사업']

In [ ]:
pt.remove('조사업')

In [ ]:
pt += new_pt

In [ ]:
pt.sort(key=len, reverse=True)

In [ ]:
pd.DataFrame(pt).to_csv('data/pre_toks_220323_1.csv', index=False)

### Test set

In [ ]:
pt = list(pd.read_csv('data/pre_toks_220323_1.csv')['0'])

In [ ]:
df_test = pd.read_csv('data/testset.csv', index_col='AI_id')

In [ ]:
df_test

,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
AI_id,,,,,,
id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...
id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매


In [ ]:
tokenizer = DealTokenizer('data/pre_toks_220323_2.csv')

In [ ]:
df_test['token_obj'] = df_test.text_obj.apply(tokenizer.tokenize)

In [ ]:
df_test['token_mthd'] = df_test.text_mthd.apply(tokenizer.tokenize)

In [ ]:
df_test['token_deal'] = df_test.text_deal.apply(tokenizer.tokenize)

In [ ]:
df_test.to_csv('data/tokenized_testset.csv')

In [ ]:
with open('data/tokenized_testset.pkl', 'wb') as f:
    pickle.dump(df_test, f)

In [ ]:
token_obj_cnt = df_test.token_obj.explode().value_counts()

In [ ]:
token_obj_cnt.to_csv('data/test_token_obj_cnt.csv')

In [ ]:
token_mthd_cnt = df_test.token_mthd.explode().value_counts()

In [ ]:
token_mthd_cnt.to_csv('data/test_token_mthd_cnt.csv')

In [ ]:
token_deal_cnt = df_test.token_deal.explode().value_counts()

In [ ]:
token_deal_cnt.to_csv('data/test_token_deal_cnt.csv')

In [ ]:
test_all_tokens = list(token_obj_cnt.index) + list(token_mthd_cnt.index) + list(token_deal_cnt.index)

In [ ]:
test_all_tokens = list(set(test_all_tokens))

In [ ]:
len(test_all_tokens)

20365

In [ ]:
'물건' in pt

True

In [ ]:
new_pt = ['강사','성발달장애','발달','장애','캠프용','캠프','남대문','약국','하는','다방','당뇨병','당뇨','신약','아이','도면','출력','미래','세대','확인','편리','입주자','입주','선주','개선','대책','응급차','응급실','응급','발주','발주처','선생님','입고','재단','제단','생삼겹살','건식','습식','수예점','수예','주민','북한','이탈','직접','물건','시청자','시청','계속','발골','처방','또는','편집기','편집실','편집','농사꾼','농사','동네','위원회']

In [ ]:
new_pt = ['섭외','의식','기부금','후원','수제품','수제','소다','연주','교육', '기름', '엿기름', '전광판용', '전광판', '필리핀', '어학연수', '굴착', '공간', '제출', '컬러풀', '컬러', '실리콘', '행동', '기호성', '기호', '소형차', '소형', '중형차', '중형', '대형차', '대형', '관한', '논문', '새우', '딱새우', '건새우', '명태', '동태전', '동태', '플라스틱용','비장애','장애','샘플','냄새','차단','자산','운용사','운용','배분','법인세','법인','노무사','노무','케어','인공지능','대금업','대금','드라마','대본','군인','전통식','전통','전통적','공산물','공산품','공산','자격증', '방과후', '돌봄', '환전업', '환전','임플란트','정원용','정원사','정원','신고자','신고','간식용','간식','뜨개용','뜨개질용','뜨개질','뜨개','중매인','중매', '인공호흡기', '해외', '국내', '국내외', '소개업', '소개', '술과안주','야구','출원']

In [ ]:
new_pt = ['센터장', '센터', '경첩', '손잡이']

In [ ]:
#to_del = ['물건']

In [ ]:
pt += new_pt

In [ ]:
#pt.remove('물건')

In [ ]:
pt = list(set(pt))

In [ ]:
pt.sort(key=len, reverse=True)

In [ ]:
len(pt)

2413

In [ ]:
pd.DataFrame(pt).to_csv('data/pre_toks_220323_2.csv', index=False)

### Refine for Komoran

In [ ]:
kom = Komoran()

In [ ]:
pt = list(pd.read_csv('data/pre_toks_220323_2.csv')['0'])

In [ ]:
len(pt)

2413

In [ ]:
for t in pt:
  if t.endswith('용'):
    print(t)
    print(kom.pos(t))

인쇄회로기판용
[('인쇄', 'NNP'), ('회로', 'NNG'), ('기판', 'NNG'), ('용', 'XSN')]
건설ㆍ광업용
[('건설', 'NNG'), ('ㆍ', 'SP'), ('광업', 'NNP'), ('용', 'XSN')]
사무ㆍ상업용
[('사무', 'NNG'), ('ㆍ', 'SP'), ('상업', 'NNP'), ('용', 'XSN')]
시스템ㆍ응용
[('시스템', 'NNG'), ('ㆍ', 'SP'), ('응용', 'NNP')]
비메모리용
[('비', 'XPN'), ('메모리', 'NNP'), ('용', 'XSN')]
플라스틱용
[('플라스틱', 'NNG'), ('용', 'XSN')]
토목공사용
[('토목', 'NNG'), ('공사', 'NNG'), ('용', 'XSN')]
공공기관용
[('공공', 'NNG'), ('기관', 'NNG'), ('용', 'XSN')]
놀이터용
[('놀이터', 'NNP'), ('용', 'XSN')]
전광판용
[('전광판', 'NNG'), ('용', 'XSN')]
자동차용
[('자동차', 'NNG'), ('용', 'XSN')]
메모리용
[('메모리', 'NNG'), ('용', 'XSN')]
농림업용
[('농림', 'NNP'), ('업', 'NNG'), ('용', 'XSN')]
뜨개질용
[('뜨개질', 'NNP'), ('용', 'XSN')]
음식점용
[('음식점', 'NNG'), ('용', 'XSN')]
비거주용
[('비', 'XPN'), ('거주', 'NNG'), ('용', 'XSN')]
스포츠용
[('스포츠', 'NNG'), ('용', 'XSN')]
비주거용
[('비', 'XPN'), ('주거', 'NNP'), ('용', 'XSN')]
항공기용
[('항공기', 'NNG'), ('용', 'XSN')]
여성용
[('여성', 'NNG'), ('용', 'XSN')]
전투용
[('전투', 'NNG'), ('용', 'XSN')]
유아용
[('유아', 'NNG'), ('용', 'XSN')]
생활용
[('생활', 'NNG'), ('

In [ ]:
pt = list(filter(lambda t: not t.endswith('용'), pt))

In [ ]:
len(pt)

2298

In [ ]:
for t in pt:
  for word, pos in kom.pos(t):
    if pos == 'NA':
      print(f't: {t}, word: {word}')

t: 드라이크리닝, word: 드라이크리닝
t: 크라프트지, word: 크라프트지
t: 엑기스, word: 엑기스
t: 돈까스, word: 돈까스
t: 쭈꾸미, word: 쭈꾸미
t: 도너츠, word: 도너츠
t: 씽크대, word: 씽크대
t: 변압, word: 변압
t: 유압, word: 유압
t: 썬팅, word: 썬팅
t: 자켓, word: 자켓
t: 압연, word: 압연
t: 압형, word: 압형
t: 압출, word: 압출


## Build dictionary

In [ ]:
tokenizer = DealTokenizer('data/pre_toks_220323_2.csv')

In [ ]:
df_test = pd.read_csv('data/testset.csv', index_col='AI_id')

In [ ]:
df_test['token_obj'] = df_test.text_obj.apply(tokenizer.tokenize)

In [ ]:
df_test['token_mthd'] = df_test.text_mthd.apply(tokenizer.tokenize)

In [ ]:
df_test['token_deal'] = df_test.text_deal.apply(tokenizer.tokenize)

In [ ]:
doc = list(df_test['token_obj']) + list(df_test['token_mthd']) + list(df_test['token_deal'])

In [ ]:
dictionary = Dictionary(doc)

In [ ]:
len(dictionary)

20167

In [ ]:
dictionary.filter_extremes(no_below=2, no_above=1.0)

In [ ]:
len(dictionary)

8107

In [ ]:
dictionary.save('data/token_dictionary_220326.dic')

In [ ]:
type(np.nan)

float

# test

In [ ]:
okt = Okt()

In [ ]:
kom = Komoran()

In [ ]:
kom.pos('본뜬치아에')

[('본뜨', 'VV'), ('ㄴ', 'ETM'), ('치아', 'NNP'), ('에', 'JKB')]

In [ ]:
okt.pos('본뜬치아에')

[('본뜬치아에', 'Verb')]

In [ ]:
kom.set_user_dictionary()

AttributeError: ignored